In [1]:
import pyspark
from random import randint, random, choice
import matplotlib.pyplot as plt

sc = pyspark.SparkContext('local[*]')

In [10]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(intGenerator(),min=1,max=100,keyName="userId")
dataset.addGenerator(intGenerator(),min=1,max=1000,keyName="songId")
dataset.addGenerator(intGenerator(),min=1,max=180,keyName="lengthPlayed")
dataset.addGenerator(dateGenerator(),startDate=datetime(2021,3,1),keyName="ts")

dataset2 = datasetGenerator()
dataset2.addGenerator(intGenerator(),min=1,max=1000,keyName="songId")
dataset2.addGenerator(intGenerator(),min=1,max=180,keyName="length")

replays = sc.parallelize(dataset.generateDataset(1000000)).persist()
metadata = sc.parallelize(dataset2.generateDataset(100000)).persist()

In [16]:
metafixed = metadata.map(lambda x: (x.get('songId'), x.get('length'))).reduceByKey(lambda a,b: a)
metafixed.first()

(804, 47)

In [22]:
joined =replays.map(lambda x: (x.get('songId'), x)).join(metafixed).map(lambda x: ((x[0], x[1][0].get('userId')), x[1][0].get('lengthPlayed'), x[1][1]))
joined.first()

((456, 97), 85, 143)

In [23]:
filtered = joined.filter(lambda x: x[1]/x[2] > 0.8)
filtered.first()

((456, 99), 133, 143)